In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize
from typing import Union
from sklearn import linear_model, covariance

In [169]:
class Analysts:
  def __init__(self) -> None:
    # Get ratings data
    self.ratings = pd.read_sas('analyst_ratings.sas7bdat', encoding='latin-1')
    self.ratings = self.ratings[['OFTIC', 'ANNDATS', 'AMASKCD', 'IRECCD']]
    self.ratings = self.ratings[self.ratings['IRECCD'] != '3']
    self.ratings['IRECCD'] = self.ratings['IRECCD'] > "3"
    self.ratings.columns = ['OFTIC', 'DATE', 'AMASKCD', 'IRECCD']
  
    # Get full historical data (Large File)
    self.hist_data = pd.read_sas('hist_price.sas7bdat', encoding='latin-1')
    self.hist_data = self.hist_data[['DATE', 'TICKER', 'PRC']]
    self.hist_data = self.hist_data.dropna()
    self.hist_data = self.hist_data.drop_duplicates(subset=['DATE', 'TICKER'])
    # Process the data to prepare for merge
    self.hist_data = self.hist_data.pivot(columns='TICKER', index='DATE', values='PRC')
    self.hist_data = self.hist_data.resample('1d').mean().ffill()
    
    # Preprocess historical data to identify growth or decline (monthly)
    self.bool_growth = (self.hist_data - self.hist_data.shift(periods=-30)).applymap(lambda x: np.NaN if np.isnan(x) else x > 0)
    self.bool_growth = self.bool_growth.melt(var_name='OFTIC',value_name='IS_GROW', ignore_index=False)
    self.bool_growth.dropna(inplace=True)
    self.bool_growth.reset_index(names=['DATE'], inplace=True)
    
    # Merge the two datasets
    self.merged = pd.merge(self.ratings, self.bool_growth, on=['OFTIC', 'DATE'], how='inner')
  
  def rank(self, end_date: Union[str, None] = None):
    """Get confidence levels without time bias - confidence prior to end_date"""
    data = self.merged
    if end_date is not None:
      data = self.merged[self.merged['DATE'] <= end_date]
    grouped = data.groupby('AMASKCD')
    analysts = {'AMASKCD': [], 'CONFIDENCE': []}
    for name, group in grouped:
      analysts['AMASKCD'].append(name)
      conf = (group['IRECCD'] == group['IS_GROW']).sum() / len(group)
      if len(group) <= 1:
        conf = 0.5
      analysts['CONFIDENCE'].append(conf)
    return pd.DataFrame(analysts)
    

In [74]:
nasdaq100 = open('nasdaq100.txt', 'r').read().split('\n')
dj30 = open('dj30.txt', 'r').read().split('\n')
train_data = nasdaq100
train_data.sort()

In [170]:
analyst = Analysts()

In [173]:
analyst_conf = analyst.rank()

In [162]:
pt = pd.read_sas('price_targets.sas7bdat', encoding='latin-1')

In [163]:
pt_masked = pt[['OFTIC', 'ANNDATS', 'AMASKCD', 'CURR', 'ESTCUR', 'HORIZON', 'VALUE']]
pt_masked = pt_masked[(pt_masked['CURR'] == 'USD') & (pt_masked['ESTCUR'] == 'USD')]
pt_masked['HORIZON'] = pt_masked['HORIZON'].astype(int)
pt_masked = pt_masked[(pt_masked['HORIZON'] >= 1) & (pt_masked['HORIZON'] <= 12)]
pt_masked = pt_masked[['OFTIC', 'ANNDATS', 'AMASKCD', 'HORIZON', 'VALUE']]
pt_masked.columns = ['OFTIC', 'DATE', 'AMASKCD', 'HORIZON', 'VALUE']

In [164]:
pt_masked

,OFTIC,DATE,AMASKCD,HORIZON,VALUE
0,TLMR,2014-03-10,71182.0,12,16.0
1,TLMR,2014-03-09,119962.0,12,16.0
2,TLMR,2014-03-10,79092.0,12,16.0
3,TLMR,2014-03-10,80474.0,12,16.0
4,TLMR,2014-04-07,79092.0,12,16.5
...,...,...,...,...,...
2090581,ZYXI,2022-10-28,118308.0,12,21.0
2090582,ZYXI,2023-01-06,193620.0,12,18.0
2090583,ZYXI,2023-03-14,118308.0,12,20.0
2090584,ZYXI,2023-03-14,122730.0,12,27.0


In [165]:
pt_merged = pd.merge(pt_masked, analyst_conf, on=['AMASKCD'], how='left')

In [72]:
pt_merged = pd.merge(pt_masked, analyst_conf, on=['AMASKCD'], how='left')
pt_merged['CONFIDENCE'] = pt_merged['CONFIDENCE'].fillna(0.5)
pt_merged = pt_merged[pt_merged['OFTIC'].isin(train_data)]
pt_merged

,OFTIC,DATE,AMASKCD,HORIZON,VALUE,CONFIDENCE
28139,TEAM,2016-01-29,70679.0,12,26.0,0.777778
28140,TEAM,2016-02-04,18603.0,12,33.0,0.722222
28141,TEAM,2016-02-05,49685.0,12,27.0,0.611111
28142,TEAM,2016-02-11,129747.0,12,26.0,0.333333
28143,TEAM,2016-04-16,79882.0,12,30.0,0.687500
...,...,...,...,...,...,...
2018504,ZS,2023-06-16,191912.0,12,173.0,0.714286
2018505,ZS,2023-06-20,194536.0,12,169.0,0.500000
2018506,ZS,2023-06-20,165573.0,12,154.0,0.555556
2018507,ZS,2023-06-20,188920.0,12,160.0,0.500000
